In [2]:
import pandas as pd
#Import SpaCy
import spacy
import regex
from spacy import displacy
from collections import Counter
import copy
from fuzzywuzzy import process
#target_nlp = spacy.load('de_core_news_sm')
IN = "input/dev.roen.df.short.tsv"
OUT = "output/masked-levenshtein-1/dev.roen.masked.tsv"
JSON = "output/masked-levenshtein-1/dev.roen.mapping.json"
entities = ["PER", "PERSON", "ORG", "ORGANIZATION", "PRODUCT", "WORK_OF_ART", "LOC"]


In [9]:
def same_enity_mask(input_csv, output_csv, output_json, entities, lang):
#Set up dict & dataframe
    d = {}
    df = input_csv
    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
    return df

In [ ]:
def mask(input_csv, output_csv, output_json, entities, lang):
    
#Set up dict & dataframe
    d = {}
    df = input_csv

    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                entity_count+=1
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                    entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
#Export dict to json:
    return df 

# EN-DE
This is a notebook detaiolign the masking of the en de dataset at different levels

## dev data

In [ ]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")

result_dev

In [ ]:
result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])
result_dev

In [ ]:
result_dev = result_dev.sort_values(by=["error"],ascending=False)
result_dev = result_dev.head(50)
result_dev.reset_index(inplace=True,drop=True)
result_dev

In [ ]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_dev_df["original"].values.tolist())[0]
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [ ]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_dev_df.to_csv("../masked-data/masked_dev_en_de_05.tsv", index = None, header=True,sep = '\t')

total_dev_df

# Training Data

In [ ]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])

In [ ]:
result_train

In [ ]:
result_train = result_train.sort_values(by=["error"],ascending=False)
num = int(7000 * 0.05)
result_train = result_train.head(num)

result_train.reset_index(inplace=True,drop=True)
result_train

In [ ]:

maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_train_df["original"].values.tolist())[0]
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [ ]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_05.tsv", index = None, header=True,sep = '\t')

total_train_df

# same enity recogniser

In [5]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")


result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])

result_dev = result_dev.sort_values(by=["error"],ascending=False)

result_dev= result_dev[result_dev["error"]>= 0.6444].reset_index(drop=True)
result_dev

,text_a,text_b,labels,predictions,error
0,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,-4.825809,-0.340117,4.485691
1,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,-3.760660,-0.349047,3.411613
2,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...",-3.730812,-0.373251,3.357561
3,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,-3.760030,-0.449195,3.310835
4,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,-3.616978,-0.311670,3.305308
...,...,...,...,...,...
522,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,0.265726,-0.383941,0.649667
523,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...",0.335268,-0.313811,0.649078
524,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,0.376660,-0.270235,0.646895
525,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...",0.320917,-0.325518,0.646435


In [6]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        pass
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
518,211,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,"[91, 95, 92, 84, 95, 71]",88.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.265726,-0.550166
519,574,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...","[91, 95, 92, 100, 76, 92]",91.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.335268,-0.381236
520,376,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,"[100, 95, 95, 100, 96, 73]",93.166667,"[0.9278538515065428, 0.7925229148025456, 0.746...",0.376660,-0.276205
521,278,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...","[90, 70, 95, 92, 78, 100]",87.500000,"[0.506574444399416, -0.19864283100263583, 0.74...",0.320917,-0.371713


In [10]:
maskd_df = same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

In [11]:
total_dev_df.to_csv("../masked-data/masked_dev_en_de_same.tsv", index = None, header=True,sep = '\t')

total_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,NE0.,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In NE1.,In NE1.,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, NE2 w...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The NE3 tied it in the third on a triple by NE...,Die NE3 band es in der dritten auf einem Tripl...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,MNE6erweise wohnten alte südarabische Sprecher...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
1514,991,"She earned high grades, excelling in French an...","Sie verdiente hohe Noten, die sich im Französi...","[91, 80, 88, 87, 88, 88]",87.000000,"[0.5487023851101287, 0.19782346731943673, 0.45...",0.359279,-0.636871
1516,993,"Trio d'anches, or trio of reeds consists of ob...",Trio d 'anches oder Trio aus Schilf besteht au...,"[50, 50, 50, 100, 82, 72]",67.333333,"[-1.1785431840290908, -0.991575427646781, -1.1...",-0.630671,-0.198255
1518,995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,"[70, 65, 95, 100, 77, 100]",84.500000,"[-0.33598436981483737, -0.3968759801636721, 0....",0.088897,-0.259579
1520,997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,"[50, 50, 40, 100, 84, 92]",69.333333,"[-1.1785431840290908, -0.991575427646781, -1.5...",-0.484296,-0.412556


In [13]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])


In [14]:
result_train = result_train.sort_values(by=["error"],ascending=False)
result_train= result_train[result_train["error"]>= 0.6444].reset_index(drop=True)

In [ ]:
maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        pass
    
maskd_df =  same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_same.tsv", index = None, header=True,sep = '\t')

total_train_df